## Quantum machine learning on lower-dimensional single-cell RNAseq data


This notebook evaluates the following quantum machine learning models:

* Quantum Support Vector Machine (QSVC) https://qiskit-community.github.io/qiskit-machine-learning/stubs/qiskit_machine_learning.algorithms.QSVC.html
* Pegasos QSVC: https://qiskit-community.github.io/qiskit-machine-learning/stubs/qiskit_machine_learning.algorithms.PegasosQSVC.html
* Neural Networks: https://qiskit-community.github.io/qiskit-machine-learning/stubs/qiskit_machine_learning.algorithms.NeuralNetworkClassifier.html
* Variational Quantum Classifier (VQC): https://qiskit-community.github.io/qiskit-machine-learning/stubs/qiskit_machine_learning.algorithms.VQC.html


It takes as input the lower dimensional embedding of the single-cell RNAseq data with eight dimension of the melanoma minimal residual diseases sample and predicts drug-administered melanoma v/s phase II of minimal residual disease. 

In [ ]:
# ====== Base class imports ======

import numpy as np
import pandas as pd
from glob import glob
import matplotlib
import os
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set_style('dark')

# ====== Scikit-learn imports ======

from sklearn.svm import SVC
from sklearn.metrics import (
    auc,
    roc_curve,
    ConfusionMatrixDisplay,
    f1_score,
    balanced_accuracy_score,
)
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split

# ====== Qiskit imports ======

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.circuit.library import QNNCircuit
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_algorithms.utils import algorithm_globals
from qiskit.primitives import Sampler
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC, PegasosQSVC
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


## ====== Torch imports ======
import torch
import torch.nn as nn 
import pytorch_lightning as pl 
from torchmetrics.classification import F1Score
import torch.optim as optim
from lightning.pytorch.utilities.types import OptimizerLRScheduler
import torch.utils.data
from pytorch_lightning.loggers import TensorBoardLogger
import lightning, lightning.pytorch.loggers


In [2]:
# load checkpoint
ckpt_path = '/dccstor/boseukb/Q/ML/checkpoints/GSE116237_forQ_iter'
fname = os.path.basename(ckpt_path)
all_checkpoints = []
for fname in glob('/dccstor/boseukb/Q/ML/checkpoints/GSE116237_forQ_iter*/**/*.ckpt', recursive=True):
    all_checkpoints.append(fname)

In [4]:
def compute_svc(X_train, y_train, X_test, y_test, c = 1):
    svc = SVC(C=c)
    # y_train = torch.argmax(torch.tensor(y_train, dtype=torch.float32),dim=1)
    # y_test = torch.argmax(torch.tensor(y_test, dtype=torch.float32),dim=1)
    svc_vanilla = svc.fit(X_train, y_train)
    labels_vanilla = svc_vanilla.predict(X_test)
    f1_svc = f1_score(y_test, labels_vanilla, average='micro')
    
    return f1_svc
    
def compute_QSVC(X_train, y_train, X_test, y_test, encoding='ZZ', c = 1, pegasos=False):
    
    #service = QiskitRuntimeService(instance="accelerated-disc/internal/default")    
    service = QiskitRuntimeService()    
    backend = AerSimulator(method='statevector')
    algorithm_globals.random_seed = 12345

    feature_map = None

    if encoding == 'ZZ' :
        feature_map = ZZFeatureMap(feature_dimension=X_train.shape[1], 
                            reps=2, 
                            entanglement='linear')
    else: 
        if encoding == 'Z': 
            feature_map = ZFeatureMap(feature_dimension=X_train.shape[1], 
                            reps=2)
        if encoding == 'P': 
            feature_map = PauliFeatureMap(feature_dimension=X_train.shape[1], 
                            reps=2, entanglement='linear')

    sampler = Sampler() 
    fidelity = ComputeUncompute(sampler=sampler)
    Qkernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)
    f1_qsvc = QSVC(quantum_kernel=Qkernel, C=c)
    
    f1_peg_qsvc = 0
    if pegasos == True: 
        peg_qsvc = PegasosQSVC(quantum_kernel=Qkernel, C=c)
        peg_qsvc_model = peg_qsvc.fit(X_train, y_train)
        labels_peg_qsvc = peg_qsvc_model.predict(X_test)
        f1_peg_qsvc = f1_score(y_test, labels_peg_qsvc, average='micro')

    return f1_qsvc,f1_peg_qsvc

def compute_estimator_QNN(X_train, y_train, X_test, y_test, primitive: str):
    
    if primitive == 'estimator':
        # construct QNN with the QNNCircuit's default ZZFeatureMap feature map and RealAmplitudes ansatz.
        qc_qnn = QNNCircuit(num_qubits=X_train.shape[1])

        estimator_qnn = EstimatorQNN(circuit=qc_qnn)
        # QNN maps inputs to [-1, +1]
        estimator_qnn.forward(X_train[0, :], algorithm_globals.random.random(estimator_qnn.num_weights))
        # construct neural network classifier
        estimator_classifier = NeuralNetworkClassifier(estimator_qnn, optimizer=COBYLA(maxiter=100))
        # fit classifier to data
        estimator_classifier.fit(X_train, y_train)
        f1_score_estimator_qnn = estimator_classifier.score(X_test, y_test)
        return f1_score_estimator_qnn
    
    if primitive == 'sampler':
        # construct a quantum circuit from the default ZZFeatureMap feature map and a customized RealAmplitudes ansatz
        qc_sampler = QNNCircuit(ansatz=RealAmplitudes(X_train.shape[1], reps=1))
        # parity maps bitstrings to 0 or 1
        def parity(x):
            return "{:b}".format(x).count("1") % 2
        output_shape = 2  # corresponds to the number of classes, possible outcomes of the (parity) mapping
        # construct QNN
        sampler_qnn = SamplerQNN(circuit=qc_sampler, interpret=parity,output_shape=output_shape,)
        # construct classifier
        sampler_classifier = NeuralNetworkClassifier(neural_network=sampler_qnn, optimizer=COBYLA(maxiter=100))
        # fit classifier to data
        sampler_classifier.fit(X_train, y_train)
        f1_score_sampler_qnn = sampler_classifier.score(X_test, y_test)
        return f1_score_sampler_qnn

In [15]:
results_dict = {}
for iter in range(25):
    matches = [x for x in all_checkpoints if "iter"+str(iter)+"_" in x]
    #iter_num = os.path.basename(all_checkpoints[0]).split('_')[2]\n",
    x_train = np.load([x for x in matches if "train_embedding" in x][0])
    x_test = np.load([x for x in matches if "test_embedding" in x][0])
    y_train = np.load([x for x in matches if "train_target" in x][0])
    y_test = np.load([x for x in matches if "test_target" in x][0])

    f1_svc = compute_svc(x_train,
                        y_train,
                        x_test,
                        y_test,
                        c=10)
    
    f1_qsvc, f1_peg_qsvc = compute_QSVC(x_train, 
                                        y_train, 
                                        x_test,
                                        y_test,
                                        c=10,
                                        pegasos=1,
                                        )
    
    f1_qsvc= compute_QNN(x_train, 
                                        y_train, 
                                        x_test,
                                        y_test,
                                        c=10,
                                        pegasos=1,
                                        )
results_dict[iter] = [f1_svc, f1_qsvc, f1_peg_qsvc]
df = pd.DataFrame.from_dict(results_dict, orient='index')
df.to_csv('/dccstor/boseukb/Q/ML/v2/results_comparison.csv', index=False, header=['SVC', 'QSVC', 'PEGQSVC'])

0.01
0.01
